# Table of Contents
1. [Importing Libraries](#importing-libraries)
2. [Data Description](#data-description)
    * [Multiple Genres](#multiple-genres)
    * [Visualisation](#visualisation)
3. [Data Preprocessing](#data-cleaning)
4. [Data Modelling](#data-modelling)
    * [Neural Network](#neural-network)
    * [K-Nearest Neighbours](#k-nearest-neighbours)
    * [Random Forest](#random-forest)
    * [Model Comparison](#model-comparison)

## Importing Libraries <a class="anchor" id="importing-libraries"></a>

Here, we import all the necessary libraries for our work.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras_tuner as kt

from tensorflow import keras

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras import layers, metrics

## Data Description <a class="anchor" id="data-description"></a>

First, the data is loaded and basic information about the data is displayed.

In [ ]:
tracks = pd.read_csv('csvs/dataset.csv', index_col=0)

tracks.head()

Our goal is to identify and predict the genres of the song, so we try display and see how many genres are there in the dataset.

In [ ]:
print('Number of genres: {}'.format(tracks.track_genre.nunique()))

# Get a count of all genre
tracks.track_genre.value_counts()

### Multiple Genres <a class="anchor" id="multiple-genres"></a>
 
We discover that some song may have multiple genres. To improve our modelling, we will be using tracks with one genre only. 

In [ ]:
# Sort by popularity first, so when we drop duplicate we drop lower popularity
# Drop duplicate if track_name, duration_ms, artists and track_genre are all the same
tracks.sort_values(by=['popularity'],ascending=False,inplace=True)
tracks.drop_duplicates(subset=['track_name','duration_ms','artists','track_genre'],inplace=True)

# If track_name, duration_ms and artists are same, but genre is different, aggregate the genre
tracks = tracks.groupby(['track_name','duration_ms','artists'],as_index=False).agg({'track_genre':lambda x: ','.join(x),
                                                                                                  'album_name': 'first',
                                                                                                  'track_id': 'first',
                                                                                                  'popularity': 'max',
                                                                                                  'explicit': 'first',
                                                                                                  'danceability': 'first',
                                                                                                  'energy': 'first',
                                                                                                  'loudness': 'first',
                                                                                                  'speechiness': 'first',
                                                                                                  'acousticness': 'first',
                                                                                                  'instrumentalness': 'first',
                                                                                                  'liveness': 'first',
                                                                                                  'valence': 'first',
                                                                                                  'tempo': 'first',
                                                                                                  'key': 'first',
                                                                                                  'mode': 'first',
                                                                                                  'time_signature': 'first'})


# Remove all tracks with more than one genre
tracks = tracks[tracks['track_genre'].str.contains(',') == False]
tracks.track_genre.value_counts()

Additionally, any genre with less than 500 tracks does not constitute enough training and test sample, and will be removed from the dataset.

In [ ]:
# Remove all genres with less than 500 tracks, maintain all columns
tracks = tracks.groupby('track_genre').filter(lambda x: len(x) > 500)
tracks.track_genre.value_counts()

To make our modelling easier, we will limit our selection to a hand selected few genres. As much as the top 10 genre present an interesting opportunity, a cursory glance at the data shows that the top 10 genres are not very distinct from each other. Hence, we will select a few genres that are more significantly distinct from one another.

In [ ]:
genre_popularity = tracks.groupby('track_genre')['popularity'].mean()
genre_popularity.sort_values(ascending=False)

# What is the difference between pop-film, k-pop, pop? 
# And what is the difference between sad and emo?

We choose the following genre for our modelling, and remove the rest of the genres from the dataset.
- Country
- Chill
- K-Pop
- Club
- Rock-n-Roll
- Classical
- Sleep
- Electronic
- Ambient
- Opera

In [ ]:
# Retain only the genres listed above
tracks = tracks[tracks['track_genre'].isin(['country', 'chill', 'k-pop', 'club', 'rock-n-roll', 'classical', 'sleep', 'electronic', 'ambient', 'opera'])]

### Visualisation <a class="anchor" id="visualisation"></a>

Since we have our genres sorted, we can now visualise the data to see if there are any interesting patterns.

In [ ]:
# Filter out all features that are not numerical
feature_numerical = [feature for feature in tracks.columns if tracks[feature].dtype!='O']

# If a feature has less than 50 unique values, it is considered as discrete
discrete_features = [feature for feature in feature_numerical if tracks[feature].nunique()<50]
continuous_features = [feature for feature in feature_numerical if feature not in discrete_features]

We first plot grouped barcharts to see the incidence of each discrete numerical feature corresponding to each genre.

In [ ]:
for feature in discrete_features:
    g = sns.catplot(
        data=tracks, kind="count",
        x="track_genre", hue=feature,
        estimator=np.median, palette="hls"
    )

    g.set_axis_labels("Genre", "Count")
    g.set_xticklabels(rotation=45)
    g.legend.set_title(feature)

Then we plot grouped boxenplots to see the distribution of each continuous numerical feature corresponding to each genre. This helps us to identify outliers and the statistical distribution of each feature.

In [ ]:
for feature in continuous_features:
    g = sns.boxenplot(
        data=tracks, x="track_genre", y=feature,
        flier_kws=dict(linewidths=0, s=10)
    )

    g.set_xticklabels(g.get_xticklabels(), rotation=45)
    g.set(title=feature)
    plt.show()

We're also checking the density of each continuous numerical feature corresponding to each genre. This helps us visualise which genre dominates which range of values for each feature.

In [ ]:
for feature in continuous_features:
    g = sns.displot(
        data=tracks, x=feature, hue="track_genre", 
        multiple="fill", kind="kde", bw_adjust=2,
        linewidth=0.5, clip=(tracks[feature].min(), tracks[feature].max())
    )

    g.set(title=feature)
    plt.show()

We check the histogram density of each discrete numerical feature to see if there are any interesting patterns.

In [ ]:
for feature in discrete_features:
    g = sns.displot(
        data=tracks, x=feature, hue="track_genre", 
        linewidth=0.5, multiple="fill",
        discrete=True
    )

    g.set(title=feature, ylabel="Proportion", \
        xticks=range(0, tracks[feature].max() + 1))
    plt.show()

## Data Preprocessing <a class="anchor" id="data-cleaning"></a>

We start off with basic data cleaning, removing null data and removing unnecessary columns according to our EDA.

In [ ]:
# Drop the row where track_name = null
tracks.drop(tracks.index[tracks['track_name'].isnull()], inplace=True)

We will also remove Track ID from our dataset as the ID is randomly generated data. Additionally, track name, artist name and album name will be removed as well. These three category are too diverse and will be hard to generalize, even if they provide very useful information. 

We will also drop the track key, as it will present too many dimension for our model to handle.

In [ ]:
# Drop the track_id column
tracks.drop('track_id', axis=1, inplace=True)

# Drop the track_name, artists, album_name columns
tracks.drop(['track_name', 'artists', 'album_name'], axis=1, inplace=True)

# Drop the key column
tracks.drop('key', axis=1, inplace=True)

Next, we will discretize both loudness, tempo and duration_ms into 10 bins each. The exact value of these columns are not important, but their rough bins will help better inform the model.

We will also normalise the popularity columns, as they are on a different scale from the rest of the data.

In [ ]:
# Discretize the loudness column into 10 bins, normalised within 0 and 1
tracks['loudness'] = pd.cut(tracks['loudness'], 10, labels=False)
tracks['loudness'] = MinMaxScaler().fit_transform(tracks[['loudness']])

# Discretize the tempo column into 10 bins, normalised within 0 and 1
tracks['tempo'] = pd.cut(tracks['tempo'], 10, labels=False)
tracks['tempo'] = MinMaxScaler().fit_transform(tracks[['tempo']])

# Normalise the duration_ms column through the use of log transformation, then normalise within 0 and 1
tracks['duration_ms'] = np.log(tracks['duration_ms'])
tracks['duration_ms'] = MinMaxScaler().fit_transform(tracks[['duration_ms']])

# Normalise the popularity column through MinMaxScaler
tracks['popularity'] = MinMaxScaler().fit_transform(tracks[['popularity']])

# Normalise the time_signature column through MinMaxScaler
tracks['time_signature'] = MinMaxScaler().fit_transform(tracks[['time_signature']])

# Describe the dataset
tracks.describe()

Next, we make sure each of the genres has 500 sample exactly.

In [ ]:
# Drop individual rows until the number of tracks per genre is equal
tracks = tracks.groupby('track_genre').apply(lambda x: x.sample(tracks.track_genre.value_counts().min(), random_state=42).reset_index(drop=True))
tracks.track_genre.value_counts()

Then, we output the data into a csv file for possible reference.

In [ ]:
tracks.to_csv('csvs/clean_data.csv',index=False)

## Data Modelling <a class="anchor" id="data-modelling"></a>

We split the data into training and test set, with 80% of the data going into training and 20% going into test.

In [ ]:
X = tracks.drop('track_genre', axis=1)
y = tracks['track_genre']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

### Neural Network <a class="anchor" id="neural-network"></a>

We use keras_tuner to help carry out hyperparameter tuning for our neural network. We vary the amount of neuron per layer, and try dropout layer to help reduce overfitting.

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(
        layers.Dense(
            units=hp.Int('units_1st', min_value=16, max_value=256, step=8),
            activation='relu',
            input_shape=(X_train.shape[1],)
        )
    )

    if hp.Boolean("dropout_1st"):
        model.add(
            layers.Dropout(
                rate=hp.Float('dropout_rate_1st', min_value=0.1, max_value=0.5, step=0.05)
            )
        )

    model.add(
        layers.Dense(
            units=hp.Int('units_2nd', min_value=16, max_value=256, step=8),
            activation='relu'
        )
    )

    if hp.Boolean("dropout_2nd"):
        model.add(
            layers.Dropout(
                rate=hp.Float('dropout_rate_2nd', min_value=0.1, max_value=0.5, step=0.05)
            )
        )


    model.add(layers.Dense(units=10,activation='softmax'))

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=[
                    metrics.CategoricalAccuracy(name='cat_acc'),
                    'accuracy'
                ])
    
    return model

Instead of grid search, we use Hyperband, which is type of random search. This is because grid search is very computationally expensive, and Hyperband is a more efficient way to carry out random search.

In [ ]:
lbl = LabelEncoder()
lbl.fit(y_train)
ann_y_train = lbl.transform(y_train)
ann_y_test = lbl.transform(y_test)

ann_y_train = tf.keras.utils.to_categorical(ann_y_train)
ann_y_test = tf.keras.utils.to_categorical(ann_y_test)

build_model(kt.HyperParameters())

tuner = kt.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=20,
    seed=42,
    executions_per_trial=2,
    overwrite=True, # change to False if you want to resume a previous search
    directory='search',
    project_name='spotify'
)
    
tuner.search(X_train, ann_y_train, validation_split=0.1)

We take the best hyperparameter from the tuner and use it to train our neural network.

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=3)
ann_model = build_model(best_hps[0])

plot_model(ann_model, show_shapes=True, show_layer_names=True, to_file='model.png')

ann_history = ann_model.fit(X_train, ann_y_train, epochs=120, batch_size=16, validation_split=0.1)
ann_score = ann_model.evaluate(X_test, ann_y_test)
ann_cm = confusion_matrix(y_test, lbl.inverse_transform(ann_model.predict(X_test)))
ann_cr = classification_report(y_test, lbl.inverse_transform(ann_model.predict(X_test)))

print('Accuracy: {}'.format(ann_score[1]))
print('Confusion Matrix: \n{}'.format(ann_cm))
print('Classification Report: \n{}'.format(ann_cr))

### K-Nearest Neighbours <a class="anchor" id="k-nearest-neighbours"></a>

K-Nearest Neighbours is a simple algorithm that uses the distance between the data points to classify the data. We use the euclidean distance to calculate the distance between the data points.

In [ ]:
knn = KNeighborsClassifier()
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': [10, 20, 30, 40, 50],   
}

knn_grid = GridSearchCV(knn, knn_param_grid, scoring='accuracy', verbose=2, n_jobs=-1)

knn_grid.fit(X_train, y_train)
print('Best KNN Params: {}'.format(knn_grid.best_params_))

knn_pred = knn_grid.predict(X_test)

# Best param was {'leaf_size': 10, 'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}
# Run the following to prevent GridSearchCV from running again
# knn = KNeighborsClassifier(leaf_size=10, metric='manhattan', n_neighbors=7, weights='distance')
# knn.fit(X_train, y_train)
# knn_pred = knn.predict(X_test)

knn_score = accuracy_score(y_test, knn_pred)
knn_cm = confusion_matrix(y_test, knn_pred)
knn_cr = classification_report(y_test, knn_pred)

print('KNN Accuracy: {}'.format(knn_score))
print('KNN Confusion Matrix: \n{}'.format(knn_cm))
print('KNN Classification Report: \n{}'.format(knn_cr))


### Random Forest <a class="anchor" id="random-forest"></a>

We generate a random forest model with 100 trees, and use the model to predict the genre of the test set.

In [ ]:
rfc = RandomForestClassifier(random_state=42)
rfc_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 9, 11, 13, 15],
    'max_features': ['sqrt', 'log2', None],
}

rfc_grid = GridSearchCV(rfc, rfc_param_grid, scoring='accuracy', verbose=2, n_jobs=-1)

rfc_grid.fit(X_train, y_train)
print('Best RFC Params: {}'.format(rfc_grid.best_params_))

rfc_pred = rfc_grid.predict(X_test)

# Best param was {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 200}
# Run the following to prevent GridSearchCV from running again
# rfc = RandomForestClassifier(max_depth=15, max_features='sqrt', n_estimators=200, random_state=42)
# rfc.fit(X_train, y_train)
# rfc_pred = rfc.predict(X_test)

rfc_cr = classification_report(y_test, rfc_pred)
rfc_cm = confusion_matrix(y_test, rfc_pred)
rfc_score = accuracy_score(y_test, rfc_pred)

print('RFC Accuracy: {}'.format(rfc_score))
print('RFC Confusion Matrix: \n{}'.format(rfc_cm))
print('RFC Classification Report: \n{}'.format(rfc_cr))

### Model Comparison <a class="anchor" id="model-comparison"></a>

Here, we compare the classification report of the three models. We use some graphs to help visualise the comparison.

In [ ]:
def plot_cm(model, X_test, y_test):
    cm = ConfusionMatrixDisplay.from_estimator(
        model, X_test, y_test, xticks_rotation=45
    )
    cm.plot()

In [ ]:
plot_cm(ann_cm, ann_grid)
plot_cm(rfc_cm, rfc_grid)
plot_cm(knn_grid, X_test, y_test)

We also plot the accuracy, loss, F1, and precision-recall curve of the neural network model.

In [ ]:
# Plot the accuracy of the models
plt.figure(figsize=(10, 5))
plt.plot(ann_history.history['accuracy'], label='ANN')
plt.plot(ann_history.history['val_accuracy'], label='ANN Validation')
plt.axhline(y=rfc_score, color='r', linestyle='-', label='RFC')
plt.axhline(y=knn_score, color='g', linestyle='-', label='KNN')
plt.title('Accuracy of Models')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the loss of the models
plt.figure(figsize=(10, 5))
plt.plot(ann_history.history['loss'], label='ANN')
plt.plot(ann_history.history['val_loss'], label='ANN Validation')
plt.title('Loss of Models')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()